# Visualize test1_dt.py Output\n
\n
This notebook helps visualize performance and outcomes from `src/detection/test1_dt.py`.\n
\n
What it can do:\n
- Parse a terminal log (saved as a text file) to extract face counts and timing (preprocess/inference/postprocess).\n
- Plot inference timings over frames and histogram.\n
- Display recent cropped faces saved by the script to `.save/`.\n
\n
Instructions:\n
1. Copy your terminal output to a text file (e.g., `logs/terminal_test1_dt_output.txt`).\n
2. Set `LOG_PATH` below to the path of that text file.\n
3. Run the cells.\n
\n
Note: The script also writes a logger file under `runs/test_logs/`, but the YOLO speed lines are printed directly to console and may not appear in the file log.\n
\n
If matplotlib is not installed, run `pip install matplotlib` in your environment.

In [1]:
import re

import numpy as np
from pathlib import Path

LOG_PATH = Path('notebooks/test_detection.log')

def parse_detection_log(log_path: Path):
    frames = []
    confidences = []
    bboxes = []
    crops = []
    if not log_path.exists():
        return {"frames": frames, "conf": confidences, "bbox": bboxes, "crops": crops}
    with open(log_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            s = line.strip()
            m_frame = re.search(r"Frame\s+(\d+):\s+Confirmed tracks:\s+(\d+)", s)
            if m_frame:
                frames.append(int(m_frame.group(1)))
                continue
            m_tr = re.search(r"->\s+Track ID:\s+\d+,\s+Confidence:\s+([0-9.]+),\s+BBox:\s*\((\d+),\s*(\d+),\s*(\d+),\s*(\d+)\)", s)
            if m_tr:
                confidences.append(float(m_tr.group(1)))
                bboxes.append(tuple(map(int, m_tr.groups()[1:])))
                continue
            m_crop = re.search(r"Saved face crop:\s+(.+)", s)
            if m_crop:
                crops.append(m_crop.group(1))
                continue
    return {"frames": frames, "conf": confidences, "bbox": bboxes, "crops": crops}

results = parse_detection_log(LOG_PATH)
print({"frames_parsed": len(results['frames']), "conf_count": len(results['conf']), "crops": len(results['crops'])})
if results['conf']:
    arr = np.array(results['conf'])
    print({
        'mean_conf': float(arr.mean()),
        'min_conf': float(arr.min()),
        'max_conf': float(arr.max()),
        'p95_conf': float(np.percentile(arr, 95))
    })
else:
    print('No confidences parsed.')

{'frames_parsed': 0, 'conf_count': 0, 'crops': 0}
No confidences parsed.


In [2]:
try:
    import matplotlib.pyplot as plt
except ImportError:
    print("matplotlib is not installed. Run: pip install matplotlib")
    plt = None

if plt and results['inference']:
    arr = np.array(results['inference'])
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].plot(arr, marker='o', ms=3)
    axes[0].set_title('Inference time per frame (ms)')
    axes[0].set_xlabel('Frame index')
    axes[0].set_ylabel('ms')

    axes[1].hist(arr, bins=20, color='steelblue')
    axes[1].set_title('Inference time histogram (ms)')
    axes[1].set_xlabel('ms')
    axes[1].set_ylabel('count')
    plt.tight_layout()
    plt.show()
else:
    print("Skipping plots.")

KeyError: 'inference'

## Display Recent Crops from `.save/`\n
\n
The script saves rate-limited face crops as `trackId_timestamp.jpg` under `.save/`.\n
We load the most recent N crops and show them in a grid.

In [5]:
SAVE_DIR = Path('.save/')
N = 12  # number of recent crops to show\n

def list_recent_crops(save_dir: Path, n: int):
    if not save_dir.exists():
        print(f"{save_dir} not found. Run test1_dt.py to generate crops.")
        return []
    files = sorted(save_dir.glob('*.jpg'), key=lambda p: p.stat().st_mtime, reverse=True)
    return files[:n]

recent = list_recent_crops(SAVE_DIR, N)
print(f"Found {len(recent)} recent crops.")

if plt and recent:
    import cv2
    rows = int(np.ceil(len(recent) / 4))
    fig, axes = plt.subplots(rows, 4, figsize=(12, 3 * rows))
    axes = np.ravel(axes) if hasattr(axes, 'ravel') else [axes]
    for i, p in enumerate(recent):
        img = cv2.imread(str(p))
        if img is None:
            axes[i].axis('off')
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[i].imshow(img_rgb)
        axes[i].set_title(p.name)
        axes[i].axis('off')
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("No crops to display or matplotlib missing.")

.save not found. Run test1_dt.py to generate crops.
Found 0 recent crops.
No crops to display or matplotlib missing.
